In [1]:
import gym
from scipy.special import expit as sigmoid
import numpy as np
import numpy.random as npr
import time
from itertools import count
from collections import deque, namedtuple
import matplotlib.pyplot as plt
from a2c_ppo_acktr import utils
import random
from tqdm import tqdm, trange
from a2c_ppo_acktr.model import DebugAttentionBase

from heap import Heap

In [2]:
class args(object):
    bandwidth = 3
    postwidth = 3
    threshold = 0.5
    signal_split = 2
    att_num_heads = 4
    fps = 3
    eval_interval = 10
    eval_num = 250
    num_steps = 32
    ppo_epoch = 4
    value_loss_coef = 0.5
    entropy_coef = 0.01
    direction_loss_coef = 0
    lr = 1e-4
    eps = 1e-5
    max_grad_norm = 0.2
    clip_param = 0.2
    gamma = 0.99
    discount_mode = 'parallel'
    parallel_envs = 32
    quantile_truncate = 0.25

In [3]:
heap = Heap(4, 8, 1, args=args, post_process=None)

/home/f1ery/.local/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [4]:
pc = np.array(heap.postcodes)

In [5]:
np.mean(np.sqrt(np.sum((pc[:, np.newaxis] - pc[np.newaxis, :])**2, axis=-1)))

0.6134421960202786

In [6]:
EnvTuple = namedtuple('EnvTuple', ['id', 'env', 'state', 'prev_state', 'rewards'])
envs = [EnvTuple(id, gym.make('CartPole-v0'), None, None, deque(maxlen=args.eval_interval)) for id in range(args.parallel_envs)]
eval_env = EnvTuple('eval', gym.make('CartPole-v0'), None, None, deque(maxlen=args.eval_num))
fpses = deque(maxlen=500)
def iterate_env(env):
    if env.state is None:
        env = env._replace(state=env.env.reset(), prev_state=env.state)
        env.rewards.append(0)
        heap.reset(env=env.id)
    out_action, fps = heap(env.state, n=args.fps, env=env.id)
    fpses.append(fps)
    if out_action:
        action = int(out_action[0] > 0.5)
    else:
        heap.done(env=env.id)
        env = env._replace(state=None, prev_state=env.state)
        return env
    state, reward, done, _ = env.env.step(action)
    heap.reward(reward, env=env.id)
    env.rewards[-1] += reward
    if done:
        if env.rewards[-1] < 200:
            heap.done(env=env.id)
        env = env._replace(state=None, prev_state=state)
        return env
    env = env._replace(state=state, prev_state=env.state)
    return env

def eval(env, j=0):
    env.rewards.clear()
    with tqdm(total=args.eval_num) as pbar:
        while True:
            env = iterate_env(env)
            if env.state is None:
                pbar.update(1)
                pbar.set_description('Eval | Avg. reward: %0.2f' % (sum(env.rewards)/len(env.rewards)))
                if len(env.rewards) == args.eval_num:
                    break
    avg_eval_reward = sum(env.rewards)/len(env.rewards)
    max_eval_reward = max(env.rewards)
    min_eval_reward = min(env.rewards)
    try:
        avg_reward = sum([sum(env.rewards) for env in envs])/sum([len(env.rewards) for env in envs])
        max_reward = max([max(env.rewards) for env in envs])
        min_reward = min([min(env.rewards) for env in envs])
    except:
        avg_reward = float('nan')
        max_reward = float('nan')
        min_reward = float('nan')
    print('Iter: %d, Avg/Max/Min. reward: %0.1f/%0.1f/%0.1f, Avg FPS: %0.2f\nEval Avg/Max/Min. reward: %0.1f/%0.1f/%0.1f' % (j, avg_reward, max_reward, min_reward, sum(fpses)/len(fpses), avg_eval_reward, max_eval_reward, min_eval_reward))

In [ ]:
eval(eval_env, 0)
for j in count(559):
    heap.clear_memory()
    final_states = []
    with tqdm(total=args.num_steps) as pbar:
        for _ in range(args.num_steps):
            for i, env in enumerate(envs):
                envs[i] = iterate_env(env)
            pbar.update(1)
            avg_reward = sum([sum(env.rewards) for env in envs])/sum([len(env.rewards) for env in envs])
            pbar.set_description('Collect | Avg. reward: %0.2f' % avg_reward)
    next_states = {env.id : env.prev_state if env.state is None else env.state for env in envs}
    heap.update(next_states)
    if j % args.eval_interval == 0:
        eval(eval_env, j)

Eval | Avg. reward: 1.08: 100%|██████████| 250/250 [00:21<00:00, 11.83it/s]
  0%|          | 0/32 [00:00<?, ?it/s]

Iter: 0, Avg/Max/Min. reward: nan/nan/nan, Avg FPS: 1.88
Eval Avg/Max/Min. reward: 1.1/13.0/0.0


Collect | Avg. reward: 1.19: 100%|██████████| 32/32 [00:39<00:00,  1.23s/it]
Updating: 100%|██████████| 13/13 [05:34<00:00, 25.72s/it]
Collect | Avg. reward: 3.73: 100%|██████████| 32/32 [00:43<00:00,  1.35s/it]
Updating: 100%|██████████| 13/13 [04:39<00:00, 21.48s/it]
Eval | Avg. reward: 18.32: 100%|██████████| 250/250 [03:05<00:00,  1.35it/s]


Iter: 560, Avg/Max/Min. reward: 3.7/34.0/0.0, Avg FPS: 0.18
Eval Avg/Max/Min. reward: 18.3/82.0/0.0


Collect | Avg. reward: 6.76: 100%|██████████| 32/32 [00:40<00:00,  1.26s/it]
Updating: 100%|██████████| 13/13 [04:25<00:00, 20.42s/it]
Collect | Avg. reward: 9.69: 100%|██████████| 32/32 [00:40<00:00,  1.28s/it]
Updating: 100%|██████████| 13/13 [04:29<00:00, 20.71s/it]
Collect | Avg. reward: 12.52: 100%|██████████| 32/32 [00:39<00:00,  1.23s/it]
Updating: 100%|██████████| 13/13 [04:20<00:00, 20.01s/it]
Collect | Avg. reward: 15.09: 100%|██████████| 32/32 [00:39<00:00,  1.25s/it]
Updating: 100%|██████████| 13/13 [04:21<00:00, 20.11s/it]
Collect | Avg. reward: 16.84: 100%|██████████| 32/32 [00:40<00:00,  1.25s/it]
Updating: 100%|██████████| 13/13 [04:22<00:00, 20.23s/it]
Collect | Avg. reward: 18.18: 100%|██████████| 32/32 [00:40<00:00,  1.28s/it]
Updating: 100%|██████████| 13/13 [04:28<00:00, 20.62s/it]
Collect | Avg. reward: 18.14: 100%|██████████| 32/32 [00:40<00:00,  1.27s/it]
Updating: 100%|██████████| 13/13 [04:25<00:00, 20.45s/it]
Collect | Avg. reward: 18.00: 100%|██████████| 32/

...

Collect | Avg. reward: 43.83: 100%|██████████| 32/32 [00:36<00:00,  1.13s/it]
Updating: 100%|██████████| 13/13 [03:57<00:00, 18.26s/it]
Collect | Avg. reward: 43.16: 100%|██████████| 32/32 [00:35<00:00,  1.12s/it]
Updating: 100%|██████████| 13/13 [03:55<00:00, 18.14s/it]
Collect | Avg. reward: 43.85: 100%|██████████| 32/32 [00:35<00:00,  1.12s/it]
Updating: 100%|██████████| 13/13 [03:55<00:00, 18.13s/it]
Collect | Avg. reward: 44.27: 100%|██████████| 32/32 [00:36<00:00,  1.13s/it]
Updating: 100%|██████████| 13/13 [03:56<00:00, 18.23s/it]
Collect | Avg. reward: 43.59: 100%|██████████| 32/32 [00:36<00:00,  1.13s/it]
Updating: 100%|██████████| 13/13 [03:57<00:00, 18.27s/it]
Collect | Avg. reward: 44.51: 100%|██████████| 32/32 [00:36<00:00,  1.13s/it]
Updating: 100%|██████████| 13/13 [03:58<00:00, 18.33s/it]
Collect | Avg. reward: 45.04: 100%|██████████| 32/32 [00:36<00:00,  1.15s/it]
Updating: 100%|██████████| 13/13 [04:01<00:00, 18.59s/it]
Collect | Avg. reward: 44.99: 100%|██████████| 3

Iter: 1050, Avg/Max/Min. reward: 45.2/172.0/0.0, Avg FPS: 0.03
Eval Avg/Max/Min. reward: 47.0/149.0/1.0


Collect | Avg. reward: 44.56: 100%|██████████| 32/32 [00:36<00:00,  1.13s/it]
Updating: 100%|██████████| 13/13 [03:57<00:00, 18.25s/it]
Collect | Avg. reward: 43.79: 100%|██████████| 32/32 [00:36<00:00,  1.14s/it]
Updating: 100%|██████████| 13/13 [03:58<00:00, 18.38s/it]
Collect | Avg. reward: 42.92: 100%|██████████| 32/32 [00:36<00:00,  1.13s/it]
Updating: 100%|██████████| 13/13 [03:58<00:00, 18.31s/it]
Collect | Avg. reward: 41.87: 100%|██████████| 32/32 [00:36<00:00,  1.13s/it]
Updating: 100%|██████████| 13/13 [03:56<00:00, 18.18s/it]
Collect | Avg. reward: 42.51: 100%|██████████| 32/32 [00:36<00:00,  1.13s/it]
Updating: 100%|██████████| 13/13 [03:57<00:00, 18.27s/it]
Collect | Avg. reward: 41.27: 100%|██████████| 32/32 [00:36<00:00,  1.13s/it]
Updating: 100%|██████████| 13/13 [03:56<00:00, 18.16s/it]
Collect | Avg. reward: 41.33: 100%|██████████| 32/32 [00:35<00:00,  1.12s/it]
Updating: 100%|██████████| 13/13 [03:55<00:00, 18.11s/it]
Collect | Avg. reward: 41.64: 100%|██████████| 3

Iter: 1070, Avg/Max/Min. reward: 43.6/200.0/0.0, Avg FPS: 0.03
Eval Avg/Max/Min. reward: 51.2/170.0/0.0


Collect | Avg. reward: 43.79: 100%|██████████| 32/32 [00:35<00:00,  1.11s/it]
Updating: 100%|██████████| 13/13 [03:53<00:00, 18.00s/it]
Collect | Avg. reward: 44.26: 100%|██████████| 32/32 [00:36<00:00,  1.14s/it]
Updating: 100%|██████████| 13/13 [03:58<00:00, 18.36s/it]
Collect | Avg. reward: 45.19: 100%|██████████| 32/32 [00:36<00:00,  1.14s/it]
Updating: 100%|██████████| 13/13 [03:57<00:00, 18.25s/it]
Collect | Avg. reward: 45.15: 100%|██████████| 32/32 [00:35<00:00,  1.12s/it]
Updating: 100%|██████████| 13/13 [03:55<00:00, 18.11s/it]
Collect | Avg. reward: 45.01: 100%|██████████| 32/32 [00:36<00:00,  1.13s/it]
Updating: 100%|██████████| 13/13 [03:55<00:00, 18.13s/it]
Collect | Avg. reward: 44.94: 100%|██████████| 32/32 [00:35<00:00,  1.12s/it]
Updating: 100%|██████████| 13/13 [03:56<00:00, 18.19s/it]
Collect | Avg. reward: 45.54: 100%|██████████| 32/32 [00:36<00:00,  1.14s/it]


In [ ]:
heap.clear_memory(eval_env.id)

In [ ]:
eval(eval_env, -1)

In [ ]:
n = 100
heap.plot_stats(n=n)

In [ ]:
heap.clear_memory(eval_env.id)
while True:
    eval_env = iterate_env(eval_env)
    if eval_env.state is None:
        break
import torch

with torch.no_grad():
    next_value = heap(eval_env.prev_state, trace_value=True, env=eval_env.id)
_, memory = heap.get_inner_state(eval_env.id)
memory.compute_returns(torch.tensor([next_value]), 0.99)

In [ ]:
eval_env.rewards

In [ ]:
for (i, id, r, pr, rew) in zip(range(10000), memory.unit_ids, memory.returns, memory.value_preds, memory.rewards):
    print(id, r, pr, rew, i in memory.discount_marks, i in memory.done_marks)
    if i in memory.done_marks:
        break

In [ ]:
last_obs = None
req_pos = 0
cur_pos = 0

In [ ]:
for i, (id, act, obs) in enumerate(zip(memory.unit_ids, memory.actions, memory.obs)):
    if id == 12 and cur_pos <= req_pos:
        last_obs = obs
        cur_pos += 1
    from_ids = []
    if len(obs.shape) == 1:
        from_ids = ['env']
    else:
        for _obs in obs:
            from_vec = _obs[:3].numpy()
            if np.sum((from_vec - heap.postcodes[0])**2) < 1e-5:
                from_id = 0
            elif np.sum((from_vec - heap.postcodes[1])**2) < 1e-5:
                from_id = 1
            elif np.sum((from_vec - heap.postcodes[2])**2) < 1e-5:
                from_id = 2
            elif np.sum((from_vec - heap.postcodes[3])**2) < 1e-5:
                from_id = 3
            else:
                _from_ids = heap.index.get_nns_by_vector(from_vec, n=2)
                from_id = _from_ids[0] if not _from_ids[0] == id else _from_ids[1]
            from_ids.append(from_id)
    to_ids = []
    did_act = []
    if act.shape[0] == 1:
        to_ids = ['env']
        did_act = ['+']
        if sigmoid(act[0, 0]) <= args.threshold:
            did_act = ['-']
        print(act)
    else:
        for _act in act:
            _did_act = '+'
            to_vec = _act[1:4].numpy()
            _to_ids = heap.index.get_nns_by_vector(to_vec, n=2)
            to_id = _to_ids[0] if not _to_ids[0] == id else _to_ids[1]
            if sigmoid(_act[0]) <= args.threshold:
                _did_act = '-'
            to_ids.append(to_id)
            did_act.append(_did_act)
    if i in memory.discount_marks:
        print('-'*80)
    if i in memory.done_marks:
        print('x'*80)
    print(from_ids, '->', id, '->', ['%s%s' % (str(to_id), did_act[j]) for j, to_id in enumerate(to_ids)])

In [ ]:
tst = DebugAttentionBase(7, 1, args.att_num_heads)
tst.load_state_dict(heap.units[-1].actor_critic.base.state_dict())
tst(last_obs)